## Apache parquet (파케이) 파일포맷

In [1]:
import pymysql
import pandas as pd

connection = pymysql.connect(host='localhost', port=3306, db='home', \
    user='root', passwd ='arfrom', autocommit=True)

In [28]:
cursor = connection.cursor()
cursor.execute("select * from daily_price where date > '20230101'")
result = cursor.fetchall()
head =[t[0] for t in cursor.description]
df = pd.DataFrame(result, columns=head)
df

,code,date,open,high,low,close,volume,chg
0,000020,20230102,9400,9700,9340,9500,189606,0.02592
1,000020,20230103,9440,9620,9040,9200,188464,-0.03158
2,000020,20230104,9250,9420,9250,9330,66861,0.01413
3,000020,20230105,9370,9570,9350,9350,85184,0.00214
4,000020,20230106,9410,9440,9300,9410,59063,0.00642
...,...,...,...,...,...,...,...,...
29332,950220,20230110,4005,4090,3955,3960,624835,-0.01737
29333,950220,20230111,3970,4045,3965,4005,303750,0.01136
29334,950220,20230112,4015,4065,3975,4065,619555,0.01498
29335,950220,20230113,4040,4050,3940,3960,701226,-0.02583


In [31]:
# csv 파일로 저장하기
df.to_csv("../daily_price.csv",index=False)  #index는 저장대상에서 제외

In [33]:
# csv 파일을 읽어들이기
# code 칼럼은 csv 로 저장했다가 읽어들이는 과정에서 숫자앞에 0 들이 사라짐
df1 = pd.read_csv("../daily_price.csv")
print(df1.info(),'\n')
df1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29337 entries, 0 to 29336
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   code    29337 non-null  int64  
 1   date    29337 non-null  int64  
 2   open    29337 non-null  int64  
 3   high    29337 non-null  int64  
 4   low     29337 non-null  int64  
 5   close   29337 non-null  int64  
 6   volume  29337 non-null  int64  
 7   chg     29337 non-null  float64
dtypes: float64(1), int64(7)
memory usage: 1.8 MB
None 



,code,date,open,high,low,close,volume,chg
0,20,20230102,9400,9700,9340,9500,189606,0.02592
1,20,20230103,9440,9620,9040,9200,188464,-0.03158
2,20,20230104,9250,9420,9250,9330,66861,0.01413
3,20,20230105,9370,9570,9350,9350,85184,0.00214
4,20,20230106,9410,9440,9300,9410,59063,0.00642
...,...,...,...,...,...,...,...,...
29332,950220,20230110,4005,4090,3955,3960,624835,-0.01737
29333,950220,20230111,3970,4045,3965,4005,303750,0.01136
29334,950220,20230112,4015,4065,3975,4065,619555,0.01498
29335,950220,20230113,4040,4050,3940,3960,701226,-0.02583


In [35]:
# pip install fastparquet
# parquet 파일로 저장하기
import fastparquet
df.to_parquet("../daily_price.gzip", compression="gzip")

In [36]:
# parquet 파일을 읽어들이기
df1 = pd.read_parquet("../daily_price.gzip")
print(df1.info(),'\n')
df1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29337 entries, 0 to 29336
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   code    29337 non-null  object 
 1   date    29337 non-null  object 
 2   open    29337 non-null  int64  
 3   high    29337 non-null  int64  
 4   low     29337 non-null  int64  
 5   close   29337 non-null  int64  
 6   volume  29337 non-null  int64  
 7   chg     29337 non-null  float64
dtypes: float64(1), int64(5), object(2)
memory usage: 1.8+ MB
None 



,code,date,open,high,low,close,volume,chg
0,000020,20230102,9400,9700,9340,9500,189606,0.02592
1,000020,20230103,9440,9620,9040,9200,188464,-0.03158
2,000020,20230104,9250,9420,9250,9330,66861,0.01413
3,000020,20230105,9370,9570,9350,9350,85184,0.00214
4,000020,20230106,9410,9440,9300,9410,59063,0.00642
...,...,...,...,...,...,...,...,...
29332,950220,20230110,4005,4090,3955,3960,624835,-0.01737
29333,950220,20230111,3970,4045,3965,4005,303750,0.01136
29334,950220,20230112,4015,4065,3975,4065,619555,0.01498
29335,950220,20230113,4040,4050,3940,3960,701226,-0.02583


#### **csv 와 parquet 의 파일크기 비교**
> * csv 보다 훨씬 parquet 파일이 작다

In [41]:
import os
format(os.stat("../daily_price.csv").st_size,","), format(os.stat("../daily_price.gzip").st_size,",")


('1,542,110', '422,896')

#### **csv 와 parquet 의 읽어들이는 속도비교**
> * 파일이 크지 않아 별 차이가 나지 않았음
> * 빅데이터인 경우 parquet 가 파일크기도 10배 적고 속도도 10배 빠르다고 알려져 있음

In [37]:
time pd.read_csv("../daily_price.csv").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29337 entries, 0 to 29336
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   code    29337 non-null  int64  
 1   date    29337 non-null  int64  
 2   open    29337 non-null  int64  
 3   high    29337 non-null  int64  
 4   low     29337 non-null  int64  
 5   close   29337 non-null  int64  
 6   volume  29337 non-null  int64  
 7   chg     29337 non-null  float64
dtypes: float64(1), int64(7)
memory usage: 1.8 MB
CPU times: total: 31.2 ms
Wall time: 30.3 ms


In [38]:
time pd.read_parquet("../daily_price.gzip").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29337 entries, 0 to 29336
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   code    29337 non-null  object 
 1   date    29337 non-null  object 
 2   open    29337 non-null  int64  
 3   high    29337 non-null  int64  
 4   low     29337 non-null  int64  
 5   close   29337 non-null  int64  
 6   volume  29337 non-null  int64  
 7   chg     29337 non-null  float64
dtypes: float64(1), int64(5), object(2)
memory usage: 1.8+ MB
CPU times: total: 46.9 ms
Wall time: 25 ms
